In [1]:
# https://medium.com/@sonicboom8/sentiment-analysis-with-variable-length-sequences-in-pytorch-6241635ae130

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('data/sentiment-analysis.csv', error_bad_lines=False)

b'Skipping line 8836: expected 4 fields, saw 5\n'
b'Skipping line 535882: expected 4 fields, saw 7\n'


In [4]:
df.head()

,ItemID,Sentiment,SentimentSource,SentimentText
0,1,0,Sentiment140,is so sad for my APL frie...
1,2,0,Sentiment140,I missed the New Moon trail...
2,3,1,Sentiment140,omg its already 7:30 :O
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,Sentiment140,i think mi bf is cheating on me!!! ...


In [5]:
import spacy
from tqdm import tqdm, tqdm_notebook, tnrange

In [6]:
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

In [7]:
df['SentimentText'] = tqdm(df['SentimentText'].apply(lambda x: x.strip()))

100%|██████████| 1578612/1578612 [00:00<00:00, 2416388.01it/s]


In [8]:
from collections import Counter

In [ ]:
words = Counter()
for sentence in tqdm(df.SentimentText.values):
    words.update(w.text.lower() for w in nlp(sentence))

 76%|███████▌  | 1200612/1578612 [07:51<02:28, 2548.47it/s]

In [ ]:
len(words)

In [ ]:
words = sorted(words, key=words.get, reverse=True)

In [ ]:
words = ['_PAD','_UNK'] + words

In [ ]:
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}

In [ ]:
def indexer(s): 
    return [word2idx[w.text.lower()] for w in nlp(s)]

In [ ]:
df['sentimentidx'] = tqdm(df.SentimentText.apply(indexer))
df['lengths'] = tqdm(df.sentimentidx.apply(len))

In [ ]:
fig = plt.figure(figsize=(8,5))
ax = sns.distplot(df.lengths.values,kde=False);
ax.set(xlabel='Tweet Length', ylabel='Frequency')

In [ ]:
df.head()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable